# WL Kernel Hyperparameter Tuning

## Import Libraries

In [ ]:
# This cell imports all the libraries needed 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from skimage.segmentation import slic
from skimage.color import rgb2lab
from skimage.segmentation import mark_boundaries
from grakel.kernels import ShortestPath
from grakel import Graph
import scipy.io
import os
import glob
from grakel import GraphKernel
from grakel.kernels import ShortestPathAttr, SubgraphMatching, PropagationAttr
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances
import cv2
import pickle
from grakel.kernels import WeisfeilerLehman, VertexHistogram, NeighborhoodHash, WeisfeilerLehmanOptimalAssignment
import csv
import statistics

# Set the random seed
np.random.seed(42)

## Define Helper Functions

In [ ]:
def downsample_image(image:np.array)->np.array:
    """ Downsample every image in order to speed up calculations.
        We choose every second pixel -> Reduce size in half. 
    :param image: The original image of type `np.array`.
    :return: Return the same image downsampled half the size. 
    """
    return image[0::2, 0::2]


def load_image(path: str)->np.array:
    """
     Load the image from the specified path. 
     Pixel intensities take values in the range [0,255] and are represented using all 3 channels (RGB). 
     redChannel   = image_imported[:,:,0] - Red channel
     greenChannel = image_imported[:,:,1] - Green channel
     blueChannel  = image_imported[:,:,2] - Blue channel
    
    :param path: The path of the image of type `str`.
    :return image: The image to return stored as `np.array`.
    """
    image_imported = Image.open(path)
    image = np.array(image_imported)
    image = downsample_image(image)
    return image


def load_ground_truth_images()->dict:
    """ Create a dictionary with groundtruth segmentations for each image in the training set.
    
    :return groundtruth: Dictionary of the form key:[segmentation,number of segments,chosen_truth]. For each image
    we might have more than 1 ground truth segmentations so we will return the segmentation with the MINIMUM 
    number of segments. Key is going to be the number of the image. 
    """
    folder_path = "C:/Users/giwrg/Desktop/Master/Modules/Thesis/Reading Material & Data/2- Data/Data Used - 500/groundTruth/train"  
    # Construct the pattern to match image files
    image_pattern = os.path.join(folder_path, '*.mat')  
    # Use glob to find all matching image file paths
    image_paths = glob.glob(image_pattern)
    train_paths = []
    groundtruth = {}
    # Replace \ with /
    for path in image_paths:
        updated_string = path.rsplit("\\", 1)
        updated_string = "/".join(updated_string)
        train_paths.append(updated_string)

    for path in image_paths:
        mat_path = path
        # Get the file name from the path
        file_name = os.path.basename(path)
        # Remove the file extension
        file_name = os.path.splitext(file_name)[0]
        # Extract the number
        image_number = file_name.split("\\")[-1]

        mat_contents = scipy.io.loadmat(mat_path)
        num_of_groundtruths = mat_contents['groundTruth'][0].shape[0]
        num_regions = []
        for i in range(0,num_of_groundtruths):
            # Find the number of distinct elements
            num_distinct_elements = len(np.unique(mat_contents['groundTruth'][0][i][0][0][0]))
            num_regions.append(num_distinct_elements)
        max_index = num_regions.index(min(num_regions))  # TODO: Modify this accordingly
        selected_truth = mat_contents['groundTruth'][0][max_index][0][0][0]
        # Downsample true segmentation 
        selected_truth = downsample_image(selected_truth)
        groundtruth[image_number] = [selected_truth,len(np.unique(selected_truth)), max_index]
    return groundtruth
    

def superpixel_SLIC(image: np.array, n_segments:int, compactness:int) ->np.array:
    """
    Create superpixels using SLIC algorithm.
    
    
    :param image: Image of type 'np.array' having all 3 channels (RGB format)
    :param n_segments: Approximate number of superpixels of type `int`.
    :param compactness: Defines the tradeoff between space and color proximity of type `int`. 
     High m  -> more square superpixels.
     Small m -> arbitary shapes for superpixels but more sensitive to boundries.
    :return superpixels: The superpixel regions of type 'np.array'.
    """
    
    # Use lab color format
    lab = rgb2lab(image)

    # Use SLIC algorithm for superpixel segmentation
    superpixels = slic(lab, n_segments=n_segments, compactness=compactness, start_label=1)
    return superpixels



def Euclidean_distance(vector_a, vector_b):
    """ Calculate Euclidean distance between co-ordinates-vectors of 2 pixels. 
    
    :param vector_a: Co-ordinates of first node of type `list`.
    :param vector_b: Co-ordinates of second node of type `list`.
    :return result: Return the Euclidean distance of co-ordinates of type `int`. 
    """
    result = np.sqrt((vector_a[0]-vector_b[0])**2 + (vector_a[1]-vector_b[1])**2)
    return result



def get_image_number(path:str)->str:
    """ Extract image number from path. 
    :param path: The path of the image of type `str`.
    :return image_number: Return the image number of type `str`.
    """
    # Get the file name from the path
    file_name = os.path.basename(path)
    # Remove the file extension
    file_name = os.path.splitext(file_name)[0]
    # Extract the number
    image_number = file_name.split("\\")[-1]
    return image_number
    

def create_graphs(image: np.array, superpixels: np.array)->list:
    """
    Create a list of Graphs where each graph corresponds to a superpixel. Every node in each graph has
    as attributes the RGB values. Each node is connected with at-most 8 neighboors. 
    
    :param image: Image of type 'np.array' having all 3 channels (RGB format).
    :param superpixels: The superpixel regions of type 'np.array'.
    :return image_graphs: A `list` with graphs for each superpixel. Each graph is an instance of the class
    `Graph` within the Grakel library.  
    """
    num_of_superpixels = len(np.unique(superpixels)) # The number of superpixels-graphs 
    image_graphs = []
    for graph_num in range(1,num_of_superpixels+1):
        indices = np.argwhere(superpixels == graph_num)  # Indices of the pixels which are in the specific superpixel
        adj_matrix = np.zeros((len(indices), len(indices)))  # Adjacency matrix
        node_attributes = {}  # Dictionary with attributes for each vertex.
        examined_vertices = []
        vertex_index = 0
        for vector in indices:
            x_coordinate       = vector[0]
            y_coordinate       = vector[1]
            redChannel_value   = image[x_coordinate,y_coordinate,0] # Red channel pixel value 
            greenChannel_value = image[x_coordinate,y_coordinate,1] # Green channel pixel value
            blueChannel_value  = image[x_coordinate,y_coordinate,2] # Blue channel pixel value
            # Update dictionary with attributes for each vertex
            node_attributes[vertex_index] = [redChannel_value, greenChannel_value, blueChannel_value]
            if examined_vertices == []:  # This is the first vertex to be added
                examined_vertices.append(vector)
                vertex_index = vertex_index + 1
            else:
                # Update adjacency matrix
                for index, existing_vertex in enumerate(examined_vertices):
                    if Euclidean_distance(existing_vertex, vector) < 1.45:  # Use a 8-neighbourhood
                        adj_matrix[index, vertex_index] = 1
                        adj_matrix[vertex_index, index] = 1
                examined_vertices.append(vector)
                vertex_index = vertex_index + 1
        G = Graph(adj_matrix, node_labels=node_attributes)
        image_graphs.append(G)
    return image_graphs


def create_graphs_grayscale(image: np.array, superpixels: np.array)->list:
    """
    Create a list of Graphs where each graph corresponds to a superpixel. Every node in each graph has
    as label the grayscale pixel intensity value in the range [0,255]. Each node is connected with at-most 8 neighboors. 
    
    :param image: Image of type 'np.array' having all 3 channels (RGB format).
    :param superpixels: The superpixel regions of type 'np.array'.
    :return image_graphs: A `list` with graphs for each superpixel. Each graph is an instance of the class
    `Graph` within the Grakel library. 
    """
    num_of_superpixels = len(np.unique(superpixels)) # The number of superpixels-graphs 
    image_graphs = []
    for graph_num in range(1,num_of_superpixels+1):
        indices = np.argwhere(superpixels == graph_num)  # Indices of the pixels which are in the specific superpixel
        adj_matrix = np.zeros((len(indices), len(indices)))  # Adjacency matrix
        node_labels = {}  # Dictionary with labels for each vertex.
        examined_vertices = []
        vertex_index = 0
        for vector in indices:
            x_coordinate       = vector[0]
            y_coordinate       = vector[1]
            image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            pixel_intensity    = image_gray[x_coordinate,y_coordinate]
            # Update dictionary with attributes for each vertex
            node_labels[vertex_index] = pixel_intensity
            if examined_vertices == []:  # This is the first vertex to be added
                examined_vertices.append(vector)
                vertex_index = vertex_index + 1
            else:
                # Update adjacency matrix
                for index, existing_vertex in enumerate(examined_vertices):
                    if Euclidean_distance(existing_vertex, vector) < 1.45:  # Use a 8-neighbourhood
                        adj_matrix[index, vertex_index] = 1
                        adj_matrix[vertex_index, index] = 1
                examined_vertices.append(vector)
                vertex_index = vertex_index + 1
        G = Graph(adj_matrix, node_labels=node_labels)
        image_graphs.append(G)
    return image_graphs


def convert_seg_to_boundaries(seg:np.array)->np.array:
    """ Convert segmentation which uses a different number for each segment into boundries which is 
        a numpy array that contains 1 for boundry pixels. 
    
    :param seg: The segmentation in region format of type 'np.array'.
    :return result: The segmentation in boundry format of type `np.array`.
    """
    seg_padded = np.pad(seg, ((1, 0), (1, 0)), mode='constant', constant_values=seg[-1, -1])
    
    dx = cv2.Sobel(seg_padded, cv2.CV_64F, 1, 0, ksize=3)
    dy = cv2.Sobel(seg_padded, cv2.CV_64F, 0, 1, ksize=3)
    boundaries = np.abs(dx) + np.abs(dy)
    
    boundaries = boundaries[:-1, :-1]
    boundaries = cv2.threshold(boundaries, 0, 1, cv2.THRESH_BINARY)[1]
    result = boundaries.astype(np.uint8)

    return result


def evaluate(Predictions: np.array, Human: np.array)->float:
    """ Calculate the F1-score between the predicted boundries and the ground truth. 
    
    :param Predictions: The predicted boundries of type `np.array`.
    :param Human: The human labelled boundries of type `np.array`.
    :return f1score: F1 score which takes values between 0 and 1. Value is rounded to 4 decimal points and the 
    closer the value to 1 the better.
    """
    
    TP = Predictions * Human
    numFP = 0
    numFN = 0
    nrow, ncol = Predictions.shape
    for i in range(nrow):
        for j in range(ncol):
            if (Predictions[i,j] == 1) & (Human[i,j]==0):
                numFP = numFP + 1
            if (Predictions[i,j] == 0) & (Human[i,j]==1): 
                numFN = numFN + 1
    numTP = np.sum(TP)
    f1score = 2 * numTP / (2 * numTP + numFP + numFN)

    return round(f1score, 4)

# Data Transformation - Convert Images to Graphs with Node Labels

In [ ]:
#######################################################################
# Create a list with all the paths for the images in the training set #
#######################################################################

folder_path_train = "C:/Users/giwrg/Desktop/Master/Modules/Thesis/Reading Material & Data/2- Data/Data Used - 500/images/train"  
# Construct the pattern to match image files
image_pattern = os.path.join(folder_path_train, '*.jpg')  
# Use glob to find all matching image file paths
image_paths = glob.glob(image_pattern)
train_paths = []

# Print the paths for each image
for path in image_paths:
    updated_string = path.rsplit("\\", 1)
    updated_string = "/".join(updated_string)
    train_paths.append(updated_string)
    
###############################################################################
# Create a list of lists containing graphs for each image in the training set #
###############################################################################

images_to_graphs = []
original_images = []
superpixel_segmentations = []
for path in train_paths:  
    current_image = load_image(path)
    original_images.append(current_image)
    segments = superpixel_SLIC(current_image, 800, 10)
    superpixel_segmentations.append(segments)
    graphs = create_graphs_grayscale(current_image, segments)
    images_to_graphs.append(graphs)
    


# Plot Superpixels for first 5 Images 

In [ ]:
# Set the figure size
plt.figure(figsize=(25, 25))

# Create comparison plots for the images
for i, image in enumerate(original_images[0:5]):
    # Plot the original image
    plt.subplot(5, 2, 2*i+1)
    plt.imshow(image)
    plt.axis('off')
    plt.title('Original', fontsize=18)

    # Plot the segmented image
    plt.subplot(5, 2, 2*i+2)
    plt.imshow(mark_boundaries(image, superpixel_segmentations[i]))
    plt.axis('off')
    plt.title('Superpixels', fontsize=18)

# Adjust the layout and spacing
plt.tight_layout()

# Load Ground Truth Segmentations of Training Set 

In [ ]:
###############################################################################
#                      Load Ground Truth Segmentations                        #
###############################################################################
# Get dictionary of ground-truth-segmentations
ground_truth_segmentations = load_ground_truth_images()

# Hyperparameter tuning - WL with Vertex Histogram as Base Kernel

In [ ]:
t = [2,3,4,5,6,7]  # Maximum number of iterations. 
F1_max = 0
t_max = 0
F_scores = []
itteration = 0

for j in t:
    itteration = itteration + 1
    print('Itteration: ', itteration)
    print('Parameter: ', j)
    print('Calculate Kernel Matrices')
    gk = WeisfeilerLehman(normalize=False, n_iter=j)
    Kernel_matrix = []
    for image in images_to_graphs:
        current_matrix = gk.fit_transform(image)
        # Convert kernel matrix to dissimilarity matrix 
        current_matrix =  np.max(current_matrix) - current_matrix
        Kernel_matrix.append(current_matrix)
    #######################################################################
    # Perform Clustering Using Kernel Matrices and Hierrarchical Clustering
    # For the number of clusters use the same number of clusters as the ground truth so that the results are comperable
    # Get the image number which will be used as key for the groundtruth
    predicted_clusters = []
    for i in range(0,len(Kernel_matrix)):
        Image_number = get_image_number(train_paths[i])
        k = ground_truth_segmentations[Image_number][1]  # Get the chosen number of regions
        # Perform hierarchical clustering with group average
        print('Perform Clustering')
        print()
        clustering = AgglomerativeClustering(n_clusters=k, linkage='average',metric='precomputed')
        labels = clustering.fit_predict(Kernel_matrix[i])
        predicted_clusters.append(labels)
    #########################################################################
    # We need to convert the cluster labels to pixel labels
    # Each graph in images_to_graphs is assigned a cluster. However each graph is also connected to 
    # several pixels. Therefore we need to assign the graph cluster to their pixels.
    image_segmentations = []
    for i in range(0,len(Kernel_matrix)):
        current_image = load_image(train_paths[i])
        n_row, n_col, _ = current_image.shape
        image_segmentation = np.zeros((n_row, n_col))  # Final Segmentation
        num_of_superpixels = len(np.unique(superpixel_segmentations[i])) # The number of superpixels-graphs 
        for graph_num in range(1,num_of_superpixels+1):
            indices = np.argwhere(superpixel_segmentations[i] == graph_num)  # Indices of the pixels which are in the specific superpixel
            for pixel in indices:
                x = pixel[0]
                y = pixel[1]
                image_segmentation[x,y] = predicted_clusters[i][graph_num-1]

        image_segmentations.append(image_segmentation)
    ##########################################################################
    # For the evaluation of the segmentation performance F1-score is going to be used.
    # Get the image number which will be used as key for the groundtruth
    F1_scores = []
    for i in range(0,len(Kernel_matrix)):
        Image_number = get_image_number(train_paths[i])
        ground_truth               = ground_truth_segmentations[Image_number]
        # Get chosen ground truth-segmentation
        ground_truth               = ground_truth[0]

        # Convert segmentations into boundry formats
        ground_truth_boundry = convert_seg_to_boundaries(ground_truth)
        image_segmentation_boundry = convert_seg_to_boundaries(image_segmentations[i])

        # Calculate performance measures 
        F_score = evaluate(image_segmentation_boundry, ground_truth_boundry)
        F1_scores.append(F_score)
    F_scores.append(F1_scores)
    avg_F = sum(F1_scores) / len(F1_scores)
    if avg_F > F1_max:
        F1_max = avg_F
        t_max = j

            
print('The chosen parameter for WL kernel is: ')
print('t =', t_max)
print('Avg F1 Score =', F1_max)

# Write the parameters in a file
parameters = [t_max, F1_max]

# Specify the file path and name
csv_file = 'WL_parameters.csv'

# Open the CSV file in write mode
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each element of the list as a row in the CSV file
    for item in parameters:
        writer.writerow([item])

        
# Store detailed results

# Specify the file path and name
csv_file = 'WL_parameters_detailed.csv'

# Open the CSV file in write mode
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each element of the list as a row in the CSV file
    for item in F_scores:
        writer.writerow([item])
        
# Save the list as a pickle file
with open('WL_parameters_detailed.pkl', 'wb') as file:
    pickle.dump(F_scores, file)

## Create Plot with Results for Different Parameters

In [ ]:
# Load the pickle file with the results
with open('WL_parameters_detailed.pkl', 'rb') as file:
    tuning_results = pickle.load(file)
    
# Create a list which stores the parameters of each trial
labels = []
for j in t:
    labels.append('n_itter= '+str(j))

# Increase figure size
plt.figure(figsize=(10, 6))
sns.set(style='ticks', context='paper')
# Plot the results
ax = sns.boxplot(data=tuning_results)
ax.set_xticklabels(labels, rotation=90)
plt.title("F1 scores on training set using WL")


# Adding labels and legend
plt.xlabel('Parameters')
plt.ylabel('F1 score')

plt.tight_layout()

# Plot Segmentations for first 5 Images

In [ ]:
# Set the figure size
plt.figure(figsize=(25, 25))

# Create comparison plots for the images
for i, image in enumerate(original_images[0:5]):
    # Plot the original image
    plt.subplot(5, 2, 2*i+1)
    plt.imshow(image)
    plt.axis('off')
    plt.title('Original', fontsize=18)

    # Plot the segmented image
    plt.subplot(5, 2, 2*i+2)
    plt.imshow(image_segmentations[i])
    plt.axis('off')
    plt.title('Segmented Image', fontsize=18)

# Adjust the layout and spacing
plt.tight_layout()